<div style="text-align: center; font-weight: bold; font-size: 300%">MAP 412 - PC 5</div>                                                        <br />      
<div style="text-align: center; font-size: 150%">École Polytechnique</div><br />  
<div style="text-align: center; font-size: 120%">Paul Calot--Plaetevoet</div>

In [1]:
import numpy as np

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column

output_notebook(hide_banner=True)

**Indiquez le temps de préparation de ce notebook :**

# Méthode de Runge-Kutta

## La méthode RK2

**Implémenter la méthode RK2. Tester le cas $f(t,y)=y$, pour $t_0=0$, $y_0=1$ et $T=1,$.**

In [2]:
def update_rk2(f, t, y, h):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta d'ordre 2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    ynp1=y+h*f(t+h*0.5,y+h*f(t,y)*0.5)
    
    return ynp1


def rk2(f, tini, tend, nt, yini):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """
    
    h=(tend-tini)/(nt-1) # attention nt est la dimension finale du vecteur. 
                     # Or pour n-1 intervalle il y a n points, il faut donc rentrer n-1 ici 
    t0=tini
    
    y=np.zeros(nt)
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=update_rk2(f,t0,y[k-1],h)
        t0=t0+h
        
    return y

**Vérifier que l'erreur de convergence $\max_{0\leq n \leq N } |y(t_n) - y_n| = O(h^2)$, c'est à dire que la méthode converge à l'ordre 2.**

In [3]:
def f(t,y):
    return y

t0=0
y0=1
tf=1
nt=10

# un premier affichage
T=np.linspace(t0,tf,nt)
Yrk=rk2(f,t0,tf,nt,y0)
Yth=np.exp(T)
    
fig1 = figure(width=990, height=400, title="Représentation pour Rk2 avec "+ str(nt) + " points")
fig1.line(T,Yrk,color='red',legend="Par Rk2")
fig1.line(T,Yth,color="blue",legend="Courbe théorique")

fig1.legend.location = "top_left"
show(fig1)

# tracer de l'erreur max en fonction de h**2
Nt=np.array([10,30,70,100,300,700,1000]) 
Err=[]
H=[]
for n in Nt:
    T=np.linspace(t0,tf,n+1)
    Yrk=rk2(f,t0,tf,n+1,y0)  # n+1 points, n intervalles
    Yth=np.exp(T)
    Err_n=abs(Yrk-Yth)
    Err.append(np.max(Err_n))# on ne prend que le max
    H.append((tf-t0)/n)
H=np.array(H)


fig = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise sur Rk2")
fig.x(H, Err,legend="Erreur pratique",color="red")
fig.xaxis.axis_label = "pas"
fig.yaxis.axis_label = "Erreur commise"
fig.legend.location = "top_left"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici
fig.line(H,H)
fig.line(H,H**2,color="green",legend="ordre 2")
fig.line(H,H**3)
fig.line(H,H**4)


show(fig)

On voit donc que la méthode de Runge-Kutta d'ordre permet déjà une convergence très rapide (on a pris 10 points sur le premier graphe) pour la fonction f utilisée. On a de plus, comme prévue théoriquement, une convergence d'ordre 2.

## La méthode RK4

**Implémenter la méthode RK4 et vérifier numériquement sur le même exemple qu'à la question précédente.**

La fonction $rk4$ est la même que $rk2$, sauf que l'on fait appel à $updaterk4$ à la place de $updaterk2$.

In [4]:
def update_rk4(f, t, y, h):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta d'ordre 4
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    
    # calcul des ki, i = 1,2,3,4
    k1=f(t,y)
    k2=f(t+h*0.5,y+h*k1*0.5)
    k3=f(t+h*0.5,y+h*k2*0.5)
    k4=f(t+h,y+h*k3)
    
    # calcul de ynp1
    ynp1=y+h*(k1+2*k2+2*k3+k4)/6
    
    return ynp1


def rk4(f, tini, tend, nt, yini):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk4
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """  
    h=(tend-tini)/(nt-1) # attention nt est la dimension finale du vecteur. 
                     # Or pour n-1 intervalle il y a n points, il faut donc rentrer n-1 ici 
    t0=tini
    
    y=np.zeros(nt)
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=update_rk4(f,t0,y[k-1],h)
        t0=t0+h
        
    return y

**Vérifier que cette méthode est d'ordre 4.**

On copie-colle le code précédemment utilisé en l'adaptant à l'utilisation de Rk4.

In [5]:
def f(t,y):
    return y

t0=0
y0=1
tf=1
nt=10

# un premier affichage
T=np.linspace(t0,tf,nt)
Yrk=rk4(f,t0,tf,nt,y0)
Yth=np.exp(T)
    
fig1 = figure(width=990, height=400, title="Représentation pour Rk4 avec "+ str(nt) + " points")
fig1.line(T,Yrk,color='red',legend="Par Rk4")
fig1.line(T,Yth,color="blue",legend="Courbe théorique")

fig1.legend.location = "top_left"
show(fig1)

# tracer de l'erreur max en fonction de h**2
Nt=np.array([10,30,70,100,300,700,1000]) 
Err=[]
H=[]
for n in Nt:
    T=np.linspace(t0,tf,n+1)
    Yrk=rk4(f,t0,tf,n+1,y0)  # n+1 points, n intervalles
    Yth=np.exp(T)
    Err_n=abs(Yrk-Yth)
    Err.append(np.max(Err_n))# on ne prend que le max
    H.append((tf-t0)/n)
H=np.array(H)


fig = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise sur Rk4")
fig.x(H, Err,legend="Erreur pratique",color="red")
fig.xaxis.axis_label = "pas"
fig.yaxis.axis_label = "Erreur commise"
fig.legend.location = "top_left"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici

fig.line(H,H**3)
fig.line(H,H**4,color="green",legend="ordre 4")
fig.line(H,H**5)


show(fig)

On a de plus, comme prévue théoriquement, une convergence d'ordre 2.

## Tableau de Butcher 

**Bonus : Implémenter une méthode de Runge-Kutta générale explicite utilisant le tableau de Butcher. On implementera une fonction update_rk qui retourne $y_{n+1}$ étant données $f,t_n,y_n,h$ et les tableaux $A,B,C$ stockant respectivement les coefficients $(a_{ij})_{1 \leq i,j \leq p}$, $(b_j)_{j=1,...,p}$ et $(c_i)_{i=1,...,p}$ décrits dans le sujet de la PC.**

In [6]:
def update_rk(f, t, y, h, A, B, C):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta 
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    A : matrice A du tableau de Butcher (incluant les zéros)
    B : vecteur B du tableau de Butcher 
    C : vecteur C du tableau de Butcher (incluant c1 = 0)
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """

    #...
    
    return ynp1


def rk(f, tini, tend, nt, yini, A, B, C):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk
       
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    A : matrice A du tableau de Butcher (incluant les zéros)
    B : vecteur B du tableau de Butcher 
    C : vecteur C du tableau de Butcher (incluant c1 = 0)

    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """

    #...
    
    return y

**Bonus : Tester avec les tableaux de Butcher correspondant aux méthodes rk2 et rk4 déjà implémentés. Vérifier que vous obtenez les mêmes résultats sur le cas $f(t,y)=y$.**

# Un système conservatif

On considère l'oscillateur harmonique :
\begin{equation}
\left\{
\begin{aligned}
q'&=p\\
p'&=-q.
\end{aligned}
\right.
\end{equation}

Avant tout chose, on connait la solution d'un oscillateur harmonique: 
$$ q: t \rightarrow  sin(t) $$ dans notre cas et pour nos conditions initiales $y_0= \begin{bmatrix}
    0 \\ 1
  \end{bmatrix} $ à $t_0=0$. 

## Schéma d'Euler explicite et implicite

**Implémenter les schémas d'Euler explicite et implicite pour le système précédent.**

Le schéma d'Euler explicite répond à: $$ y_{n+1}=y_{n}+hf(t_n,y_{n}) \space \space \space avec \space y_{n}= [q_n,p_n]^\top \space et \space f\big(t,\begin{bmatrix}
    q \\ p
  \end{bmatrix}\big)= \begin{bmatrix}
    q' \\ p'
  \end{bmatrix} =\begin{bmatrix}
    p \\ -q
  \end{bmatrix}\space ici.$$
<br>
Pour obtenir le schéma implicite, il faut travailler un peu plus puisqu'il répond à: $$ y_{n+1}=y_{n}+hf(t_{n+1},y_{n+1}) $$
Il faut pouvoir isoler $y_{n+1}$ si nous voulons nous en sortir. On résout en $\begin{bmatrix}
    q_{n+1} \\ p_{n+1}
  \end{bmatrix}$ le système suivant:  $$\begin{bmatrix}
    q_{n+1} \\ p_{n+1}
  \end{bmatrix}=\begin{bmatrix}
    q_{n} \\ p_{n}
  \end{bmatrix}+hf(t_{n+1},\begin{bmatrix}
    q_{n+1} \\ p_{n+1}
  \end{bmatrix})= \begin{bmatrix}
    q_{n} \\ p_{n}
  \end{bmatrix}+h\begin{bmatrix}
    p_{n+1} \\ -q_{n+1} 
  \end{bmatrix}$$ <br> qui est un système linéaire de 2 équations à 2 inconnus. 
 <br><br> On fait alors l'opération suivante sur les lignes du système: $ L1 \leftarrow L1 + h L2 $ qui permet d'avoir $q_{n+1}$ dont on injecte l'expression dans $L2$:
 $$ \begin{bmatrix}
    q_{n+1} \\p_{n+1} 
  \end{bmatrix} = \begin{bmatrix}
    \frac{1}{1+h^2} (q_n + h p_n) 
    \\ \frac{1}{1+h^2} (-h q_n + (1+h^2-h^2) p_n)
  \end{bmatrix} = \frac{1}{1+h^2} \begin{bmatrix}
    1 & h \\ -h & 1 
  \end{bmatrix}  \begin{bmatrix}
    q_{n} \\ p_{n} 
  \end{bmatrix}
 = A \begin{bmatrix}
    q_{n} \\ p_{n} 
  \end{bmatrix} $$

In [7]:
def euler_explicite(f, tini, tend, nt, yini): # attention cette fois yini est de dimension 2
    h=(tend-tini)/(nt-1)
    t0=tini
    
    y=np.zeros((nt,2))
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=y[k-1]+h*f(t0,y[k-1])
        t0=t0+h

    return y
    
def euler_implicite(A, tini, tend, nt, yini): # y réfléchir au calme
    h=(tend-tini)/(nt-1)
    t0=tini
    
    y=np.zeros((nt,2))
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=np.dot(A,y[k-1]) 
        t0=t0+h

    return y

**Représenter les solutions $q_n$ et $p_n$ obtenues, ainsi que l'hamiltonien $\mathcal{H}(q_n,p_n)$ associé. On pourra tracer $q$ et $p$ en fonction du temps, mais aussi tracer la solution dans le plan de phase $(q,p)$. Commenter les résultats obtenus.**

In [8]:
def f(t,y):
    return(np.array([y[1],-y[0]]))

def H(p,q):
    return 0.5 * (p*p+q*q)

def solutionq(t):
    return np.sin(t)

def solutionp(t):
    return np.cos(t) 

t0=0
y0=np.array([0,1])
tf=10
nt=100

h=(tf-t0)/(nt-1) # attention au nt-1
A=1/(1+(h*h)) * np.array([[1,h],[-h,1]]) # définition de A en accord avec ce qui précède.

# affichage
T=np.linspace(t0,tf,nt)
Yrk=euler_explicite(f,t0,tf,nt,y0)
Yrki=euler_implicite(A,t0,tf,nt,y0)
Ythq=solutionq(T)
Ythp=solutionp(T)

fig1 = figure(width=500, height=300, title="Position par Euler explicite : "+ str(nt) + " points")
fig1.x(T,Yrk[:,0],color='red',legend="Euler Explicite")
fig1.x(T,Yrki[:,0],color='blue',legend="Euler Implicite")
fig1.x(T,Ythq,color="green",legend="solution exacte")
fig1.xaxis.axis_label = "temps"
fig1.yaxis.axis_label = "position"

fig2 = figure(width=500, height=300, title="Vitesse par Euler explicite : "+ str(nt) + " points")
fig2.x(T,Yrk[:,1],color='red',legend="Euler Explicite")
fig2.x(T,Yrki[:,1],color='blue',legend="Euler Implicite")
fig2.x(T,Ythp,color="green",legend="solution exacte")
fig2.xaxis.axis_label = "temps"
fig2.yaxis.axis_label = "Vitesse"

He=[H(y[0],y[1]) for y in Yrk]
Hi=[H(y[0],y[1]) for y in Yrki]
Hth=[0.5*(Ythq[k]**2+Ythp[k]**2) for k in range(len(Ythq))]
fig3 = figure(width=500, height=300, title="Hamiltonien par Euler explicite : "+ str(nt) + " points")
fig3.x(T,He,color='red',legend="Euler Explicite")
fig3.x(T,Hi,color='blue',legend="Euler Implicite")
fig3.x(T,Hth,color="green",legend="solution exacte")

fig3.xaxis.axis_label = "temps"
fig3.yaxis.axis_label = "hamiltonien"

fig4 = figure(width=500, height=300, title="Portrait de phase par Euler explicite : "+ str(nt) + " points")
fig4.x(Yrk[:,0],Yrk[:,1],color="blue",legend="Euler Explicite")
fig4.x(Yrki[:,0],Yrki[:,1],color="red",legend="Euler Implicite")
fig4.x(Ythq,Ythp,color="green",legend="Solution exacte")
fig4.xaxis.axis_label = "position"
fig4.yaxis.axis_label = "vitesse"

fig1.legend.location = "bottom_left"
fig2.legend.location = "bottom_left"
fig3.legend.location = "top_left"
fig4.legend.location = "bottom_left"

show(row(fig1,fig2))
show(row(fig3,fig4))


# étude de l'erreur pour plusieurs n:

Nt=10*np.array([10,30,70,100,300,700,1000]) 
ErrI=[]
ErrE=[]
for n in Nt:
    T=np.linspace(t0,tf,n+1)
    YrkE=euler_explicite(f,t0,tf,n+1,y0)  # n+1 points, n intervalles
  
    h=(tf-t0)/(n-1) # attention au nt-1
    A=1/(1+(h*h)) * np.array([[1,h],[-h,1]]) # définition de A en accord avec ce qui précède.
    YrkI=euler_implicite(A,t0,tf,n+1,y0)  
    
    Yth=solutionq(T)
    
    errE=abs(YrkE[:,0]-Yth)
    ErrE.append(np.max(errE))# on ne prend que le max (sera tout le temps le dernier)
    
    errI=abs(YrkI[:,0]-Yth)
    ErrI.append(np.max(errI))



fig5 = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise sur la méthode d'Euler")
fig5.x(Nt, ErrE,legend="Erreur pratique - explicite",color="red")
fig5.x(Nt, ErrI,legend="Erreur pratique - implicite",color="blue")
fig5.xaxis.axis_label = "nombre de subdivisions"
fig5.yaxis.axis_label = "Erreur commise sur la position"
fig5.legend.location = "top_left"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici

fig5.line(Nt,1/Nt,legend="Ordre 1")
fig5.legend.location = "top_right"


show(fig5)

On remarque que les deux méthodes ne proposent pas des convergence très satisfaisantes. Sur le dernier graphe, on confirme la convergence d'ordre 1 (cela dit pas à un nombre de subdivisions trop faible devant les variations de la fonction pour l'intervalle considéré, il faut toujours un mimimum de points. Cela fait penser au critère de Shannon en physique).

L'hamiltonien n'est conservé pour aucun des deux schéma. Pour Euler-explicite, on trouve une croissance qui est confortée par l'expression trouvée par le calcul: $H_{n+1}=(1+h^2) H_n$ contre $H_{n+1}=\frac{1}{1+h^2} H_n$ pour le schéma Euler-explicite, ce qui conforte la décroissance.

Afin de tester ces expressions théoriques et la validité de nos résultats, nous allons faire varier le pas et étudier numériquement la dérive de $H_n$ par rapport à $H_0=0.5$ (pour nos conditions initiales), cela donnant donc: $H_{n}=(1+h^2)^n H_0$, donc avec $h=\frac{1}{n^2}$ $H_{n}-H_0=((1+\frac{1}{n^2})^n -1) H_0 \sim \frac{H_0}{n} $

In [9]:
Nt=10*np.array([10,30,70,100,300,700,1000]) 
ErrE=[]
ErrI=[]
Hv=[]
H0=0.5 # avec les conditions initiales
for n in Nt:
    T=np.linspace(t0,tf,n+1)
    h=(tf-t0)/n
    A=1/(1+(h*h)) * np.array([[1,h],[-h,1-h*h]]) # définition de A en accord avec ce qui précède.
    
    Yrk=euler_explicite(f,t0,tf,n+1,y0)
    Yrki=euler_implicite(A,t0,tf,n+1,y0)
    Yth=np.exp(T)
    
    ye,yi=Yrk[-1],Yrki[-1]
    
    errE=abs(H(ye[0],ye[1])-H0) # H le hamiltonien
    errI=abs(H(yi[0],yi[1])-H0)
    
    ErrE.append(errE)
    ErrI.append(errI)
    
    Hv.append(h)
    
Hv=np.array(Hv)


fig = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise sur les méthodes d'Euler")
fig.x(Nt, ErrE,legend="Erreur pratique - euler explicite",color="red")
fig.x(Nt, ErrI,legend="Erreur pratique - euler implicite",color="blue")
fig.xaxis.axis_label = "nombre de subdivisions"
fig.yaxis.axis_label = "Erreur commise"
fig.legend.location = "top_right"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici

fig.line(Nt,1/Nt)
#fig.line(Hv,Hv**2,color="green",legend="ordre 2")
#fig.line(Hv,Hv**3)


show(fig)

On est donc très content d'observer une décroissance d'ordre 1 par rapport au nombre de subdivisions. 

## Schéma d'ordre élevé

**Appliquer le schéma RK2 (décrit dans l'exercice précédent) au système de l'ocillateur harmonique. Comparer les résultats $p_n$, $q_n$ et $\mathcal{H}(q_n,p_n)$ avec ceux obtenus à la question précédente. Commenter les résultats obtenus.***

# A faire: mettre sur un même graphe avec ce qui précède pour pouvoir comparer 

In [10]:
def update_rk2(f, t, y, h):
    """retourne l'approximation yn+1 de y au temps tn+1 = tn + h avec une méthode de Runge-Kutta d'ordre 2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    t : temps tn
    y : valeur de yn
    h : pas de temps
    
    Returns
    -------
    ynp1 : valeur de yn+1
    """
    ynp1=y+h*f(t+h*0.5,y+h*f(t,y)*0.5)
    
    return ynp1


def rk2(f, tini, tend, nt, yini):
    """retourne le vecteur contenant les valeurs des yn en appelant update_rk2
    
    Parameters
    ----------
    f : fonction f(t,y) second membre de l'EDO
    tini : temps initial
    tend : temps final
    nt : dimension du vecteur de sortie
    yini : valeur initiale y(tini)
    
    Returns
    -------
    y : vecteur les nt valeurs yn
    """
    
    h=(tend-tini)/(nt-1) # attention nt est la dimension finale du vecteur. 
                     # Or pour n-1 intervalle il y a n points, il faut donc rentrer n-1 ici 
    t0=tini
    
    y=np.zeros((nt,2))
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=update_rk2(f,t0,y[k-1],h)
        t0=t0+h
        
    return y

In [11]:
def f(t,y):
    return(np.array([y[1],-y[0]]))

def H(p,q):
    return 0.5 * (p*p+q*q)

t0=0
y0=np.array([0,1])
tf=10
nt=100


T=np.linspace(t0,tf,nt)
Yrk=euler_explicite(f,t0,tf,nt,y0)
Yrki=rk2(f,t0,tf,nt,y0)
Ythq=solutionq(T)
Ythp=solutionp(T)

fig1 = figure(width=500, height=300, title="Position : "+ str(nt) + " points")
fig1.x(T,Yrk[:,0],color='red',legend="Euler Explicite")
fig1.x(T,Yrki[:,0],color='blue',legend="Rk2")
fig1.x(T,Ythq,color="green",legend="solution exacte")
fig1.xaxis.axis_label = "temps"
fig1.yaxis.axis_label = "position"

fig2 = figure(width=500, height=300, title="Vitesse: "+ str(nt) + " points")
fig2.x(T,Yrk[:,1],color='red',legend="Euler Explicite")
fig2.x(T,Yrki[:,1],color='blue',legend="Rk2")
fig2.x(T,Ythp,color="green",legend="solution exacte")
fig2.xaxis.axis_label = "temps"
fig2.yaxis.axis_label = "Vitesse"

He=[H(y[0],y[1]) for y in Yrk]
Hi=[H(y[0],y[1]) for y in Yrki]
Hth=[0.5*(Ythq[k]**2+Ythp[k]**2) for k in range(len(Ythq))]
fig3 = figure(width=500, height=300, title="Hamiltonien : "+ str(nt) + " points")
#fig3.x(T,He,color='red',legend="Euler Explicite")
fig3.x(T,Hi,color='blue',legend="Rk2")
fig3.x(T,Hth,color="green",legend="solution exacte")

fig3.xaxis.axis_label = "temps"
fig3.yaxis.axis_label = "hamiltonien"

fig4 = figure(width=500, height=300, title="Portrait de phase : "+ str(nt) + " points")
fig4.x(Yrk[:,0],Yrk[:,1],color="blue",legend="Euler Explicite")
fig4.x(Yrki[:,0],Yrki[:,1],color="red",legend="Rk2")
fig4.x(Ythq,Ythp,color="green",legend="Solution exacte")
fig4.xaxis.axis_label = "position"
fig4.yaxis.axis_label = "vitesse"

fig1.legend.location = "bottom_left"
fig2.legend.location = "bottom_left"
fig3.legend.location = "top_left"
fig4.legend.location = "top_right"

show(row(fig1,fig2))
show(row(fig3,fig4))


# étude de l'erreur pour plusieurs n:

Nt=np.array([10,30,70,100,300,700,1000]) 
Err=[]

for n in Nt:
    T=np.linspace(t0,tf,n+1)
    Yrk=rk2(f,t0,tf,n+1,y0)  # n+1 points, n intervalles
  
    Yth=solutionq(T)
    
    err=abs(Yrk[:,0]-Yth)
    Err.append(np.max(err))# on ne prend que le max (sera tout le temps le dernier)
  


fig5 = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise par Rk2")
fig5.x(Nt, Err,legend="Erreur pratique - Rk2",color="red")

fig5.xaxis.axis_label = "pas"
fig5.yaxis.axis_label = "Erreur commise sur la position"
fig5.legend.location = "top_left"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici
Ht=1/Nt
fig5.line(Nt,Ht)
fig5.line(Nt,Ht**2,color="green",legend="ordre 2")
fig5.line(Nt,Ht**3)
fig5.legend.location = "top_right"


show(fig5)

On trouve un schéma plus stable que les précédents avec une convergence de l'erreur en ordre 2. Cependant, comme on le voit sur le graphe du hamiltonien, la divergence de celui-ci est toujours présente bien que beaucoup moins importante. 
NB: je n'ai pas affiché contrairement aux autre graphes le schéma Euler explicite comme il empêchait de distinguer la divergence de Rk2 vis-à-vis du Hamiltonien.

## Schéma symplectique

Considérez le schéma suivant :

\begin{equation*}
  \label{schema_symp}
  y_{n+1} = y_n + h f\left(t_n+\frac{h}{2},\ \frac{y_n+y_{n+1}}{2}\right).
\end{equation*}

**Implémenter ce schéma pour le système  l'ocillateur harmonique.**

In [12]:
def pt_milieu_symplectique(A, tini, tend, nt, yini): # attention cette fois yini est de dimension 2
    h=(tend-tini)/(nt-1)
    t0=tini
    
    y=np.zeros((nt,2))
    y[0]=yini
    
    for k in range(1,nt):
        y[k]=np.dot(A,y[k-1]) 
        t0=t0+h

    return y



**Observer numériquement son ordre, ainsi que l'évolution de l'hamiltonien, et commenter les résultats obtenus.**

Pour notre fonction on trouve: 
 $$ \begin{bmatrix}
    q_{n+1} \\p_{n+1} 
  \end{bmatrix} = \begin{bmatrix}
    \frac{1}{1+\frac{h^2}{2}} (q_n (1-\frac{h^2}{4}) + h p_n) 
    \\ \frac{1}{1+\frac{h^2}{4}} ( -h q_n + (1-\frac{h^2}{4} ) p_n
  \end{bmatrix} = \frac{1}{1+\frac{h^2}{4}} \begin{bmatrix}
    1-\frac{h^2}{4} & h \\ -h & 1-\frac{h^2}{4} 
  \end{bmatrix}  \begin{bmatrix}
    q_{n} \\ p_{n} 
  \end{bmatrix}
 = A_1 \begin{bmatrix}
    q_{n} \\ p_{n} 
  \end{bmatrix} $$

In [15]:
def f(t,y):
    return(np.array([y[1],-y[0]]))

def H(p,q):
    return 0.5 * (p*p+q*q)

def solution(t):
    return np.sin(t)


t0=0
y0=np.array([0,1])
tf=10
nt=100

h=(tf-t0)/(nt-1) # attention au nt-1
A=1/(1+h*h/4)*np.array([[1-h*h/4,h],[-h,1-h*h/4]])

# affichage
T=np.linspace(t0,tf,nt)
Yth=solution(T)
Yrk=rk2(f,t0,tf,nt,y0)
Yrki=pt_milieu_symplectique(A,t0,tf,nt,y0)
    
fig1 = figure(width=500, height=300, title="Position Schéma symplectique : "+ str(nt) + " points")
fig1.line(T,Yth,color='green',legend="Solution exacte")
fig1.x(T,Yrk[:,0],color='blue',legend="Rk2")
fig1.x(T,Yrki[:,0],color='red',legend="Point symplectique")
fig1.xaxis.axis_label = "temps"
fig1.yaxis.axis_label = "position"

fig2 = figure(width=500, height=300, title="Vitesse Schéma symplectique : "+ str(nt) + " points")
fig2.x(T,Yrk[:,1],color='red',legend="rk2")
fig2.x(T,Yrki[:,1],color='blue',legend="Point symplectique")
fig2.xaxis.axis_label = "temps"
fig2.yaxis.axis_label = "Vitesse"

He=[H(y[0],y[1]) for y in Yrk]
Hi=[H(y[0],y[1]) for y in Yrki]
fig3 = figure(width=500, height=300, title="Hamiltonien Schéma symplectique : "+ str(nt) + " points")
fig3.x(T,He,color='red',legend="rk2")
fig3.x(T,Hi,color='blue',legend="Point symplectique")
fig3.xaxis.axis_label = "temps"
fig3.yaxis.axis_label = "hamiltonien"

fig4 = figure(width=500, height=300, title="Portrait Schéma symplectique : "+ str(nt) + " points")
fig4.x(Yrk[:,0],Yrk[:,1],color="blue",legend="rk2")
fig4.x(Yrki[:,0],Yrki[:,1],color="red",legend="Point symplectique")
fig4.xaxis.axis_label = "position"
fig4.yaxis.axis_label = "vitesse"

fig1.legend.location = "top_left"
fig2.legend.location = "top_right"
fig3.legend.location = "top_left"
fig4.legend.location = "top_right"

show(row(fig1,fig2))
show(row(fig3,fig4))

# étude de l'erreur pour plusieurs n:

Nt=10*np.array([10,30,70,100,300,700,1000]) 
Err=[]

for n in Nt:
    
    T=np.linspace(t0,tf,n+1)
    
    h=(tf-t0)/n # attention au nt-1
    A=1/(1+h*h/4)*np.array([[1-h*h/4,h],[-h,1-h*h/4]])
    Yrk=pt_milieu_symplectique(A,t0,tf,n+1,y0)  # n+1 points, n intervalles
  
    Yth=solutionq(T)
    
    err=abs(Yrk[:,0]-Yth)
    Err.append(np.max(err))# on ne prend que le max (sera tout le temps le dernier)
  


fig5 = figure(width=990, height=400,x_axis_type="log",y_axis_type="log",title="Erreur commise")
fig5.x(Nt, Err,legend="Erreur pratique - symplectique",color="red")

fig5.xaxis.axis_label = "nombre du subdivisions"
fig5.yaxis.axis_label = "Erreur commise sur la position"
fig5.legend.location = "top_left"

# tracé de quelques droites: ordres 1, 2, 3 et 4 représentés ici
Ht=1/Nt
fig5.line(Nt,Ht)
fig5.line(Nt,Ht**2,color="green",legend="ordre 2")
fig5.legend.location = "top_right"


show(fig5)

L'Hamiltonien demeure cette fois constant au cours du temps en accord avec la partie théorique, la convergence est comme pour Rk2 d'ordre 2 par rapport au nombre de subdivisions. Nous avons donc les avantages des deux.